In [1]:
import pandas as pd
import numpy as np
import pickle

from scipy.sparse import load_npz, csr_matrix
from sklearn.neighbors import NearestNeighbors
from scripts.caracteristica import caracteristica as car

from datasketch import MinHash, MinHashLSH

In [2]:
def jaccard_distance_sparse(u, v):
    intersection = u.multiply(v).sum()
    union = u.maximum(v).sum()
    return 1.0 - intersection / union

In [3]:
n_perm = 128

def minhash_vector(sparse_vector:csr_matrix, num_perm:int=n_perm):
    minhash = MinHash(num_perm=num_perm)
    for index in sparse_vector.indices:
        minhash.update(str(index).encode('utf8'))
    return minhash

In [4]:
k = 2
th = 0.2
lsh = pickle.load(open(f'../Data/lsh/k{k}_th{th}.pkl', 'rb'))

In [5]:
caracteristica, tweets = car(k=k)

In [121]:
indice = 1006

m = minhash_vector(caracteristica[indice])
aprox = lsh.query(m)

original = tweets.iloc[indice]
print(f'{original["screen_name"] + ";".ljust(10)} {original["text"]}\n')

repeticiones = {}
for i in aprox:
    twt = tweets.iloc[i]
    print( f'{(str(i) + ",").ljust(8)} {twt["screen_name"].strip().ljust(20)}:'.ljust(30), twt['text'] )
    if twt['screen_name'] not in repeticiones:
        repeticiones[twt['screen_name']] = 1
    else:
        repeticiones[twt['screen_name']] += 1

masterifr;          y el partido comunista no tiene plata por favor hugito no nos mienta mas

67587,   Camus6Camus         : @meqchile asi que no nos expropiaran nuestros fondos vieja comunista qla mentirosa 
182278,  Worges              : @t13 no nos debe extrañar aqui mario waissbluth explica el mamarracho constituyente 
1028103, keno_mesigan        : @patriciapolitz @convencioncl uy si el final es super distinto como no nos dimos cuenta
169992,  Jaimeteov           : @rmontero_ q quieres construir no nos metas a nosotros
550921,  SergioBecker14      : @alvaroacevedo @jaime_bassa cierto correcto los bolcheviques no nos engañan con sus mentiras
8205,    MarcoAmpuero6       : naitul y loncon son lo mismo no nos quieren
512015,  hernanbachler       : no nos engañemos la derecha no es tonta a kast 
1204241, ciberdine2012       : @ignacioachurra rechazoooo no nos dejaron otra 
735254,  ProfeChrispo        : que no nos metan en la cabeza que ser distintos es un problema la diversidad nos e

Seleccionamos tweets de original y el usuario mas repetido.

In [122]:
mas_repetido = max(repeticiones)
tweets = pd.read_parquet('../Data/tweets.parquet')
# par = []
par_con_nombres = []
par_sin_nombres = []
for c, name in enumerate(tweets['screen_name']):
    if name == original['screen_name']:
        tweet = tweets['text'].iloc[c]
#         par.append(f'{name}: {tweet}')
        par_con_nombres.append((name, tweet))
        par_sin_nombres.append(tweet)
    elif name == mas_repetido:
        tweet = tweets['text'].iloc[c]
#         par.append(f'{name}: {tweet}')
        par_con_nombres.append((name, tweet))
        par_sin_nombres.append(tweet)

Funcion que calcula diferencia jaccard entre los tweets, se la pedi a chatgpt asique igual seria buena idea revisarla en mas detalle.

In [123]:
# import textdistance
# import heapq

# def find_most_similar_pairs(strings, k=30):
#     pairs = []
#     heap = []
    
#     for i in range(len(strings)):
#         for j in range(i + 1, len(strings)):
#             similarity_score = textdistance.jaccard(strings[i], strings[j])
            
#             heapq.heappush(heap, (similarity_score, (strings[i], strings[j])))
#             if len(heap) > k:
#                 heapq.heappop(heap)
    
#     while heap:
#         pairs.append(heapq.heappop(heap)[1])
    
#     return pairs[::-1]

def shingles(k, words: np.ndarray):
    shingles = []
    for i in range(0, len(words)):
        shing = ' '.join(words[i:i+k])
        if len(shing.split()) == k:
            shingles.append(shing)
    return list(set(shingles))


shings_list = []
for twt in par_sin_nombres:
    twt_shings = shingles(2, np.array(twt.split()))
    if twt_shings != []:
        for shing in twt_shings:
            shings_list.append(shing)
            
shings_list

['no nos',
 'partido comunista',
 'por favor',
 'nos mienta',
 'plata por',
 'mienta mas',
 'y el',
 'no tiene',
 'favor hugito',
 'hugito no',
 'el partido',
 'tiene plata',
 'comunista no',
 'por rechazo',
 'voy por',
 'siguen apoyando',
 'yo ahora',
 'apoyando a',
 'los delincuentes',
 'a los',
 'delincuentes con',
 'sus aprobaciones',
 'ahora voy',
 'con sus',
 'aprobaciones por',
 'constituyentes siguen',
 'por esto',
 'los constituyentes',
 'esto yo',
 'una de',
 'de un',
 'yo cambiaria',
 'por una',
 'la fotito',
 'de abajo',
 'un circo',
 '@rkatrileo yo',
 'fotito de',
 'cambiaria la',
 'abajo por',
 '@fernando_atria don',
 'refundar chile',
 'que quiere',
 'quiere refundar',
 'atria el',
 'fernando atria',
 'don fernando',
 'marxista que',
 'el titiritero',
 'titiritero marxista',
 'votar rechazo',
 'a votar',
 'claro a',
 'mas claro',
 'q mas',
 'cuando estos',
 'adoctrinados gatitos',
 'generando odio',
 'hasta cuando',
 'entre chilenos',
 'siguen generando',
 'chalet siguen

Calculamos los tweets mas similares, y mantenemos solamente los pares en donde haya un tweet de usuario original y otro del mas repetido.

In [124]:
# similar_pairs = find_most_similar_pairs(par)
# for pair in similar_pairs:
#     og = 0
#     repetido = 0
#     for tweet in pair:
#         if mas_repetido in tweet:
#             repetido += 1
#         elif original['screen_name'] in tweet:
#             og += 1
#     if og == 1 and repetido == 1:
#         print(pair)

def create_characteristic_matrix(list1, list2):
    matrix = []
    for string1 in list1:
        row = []
        for string2 in list2:
            if string1 in string2:
                row.append(1)
            else:
                row.append(0)
        matrix.append(row)
    return matrix

v = create_characteristic_matrix(shings_list, par_sin_nombres)

def calculate_jaccard_difference(matrix, s):
    num_cols = len(matrix[0])
    jaccard_matrix = []

    for i in range(num_cols):
        for j in range(i + 1, num_cols):
            intersection = np.sum(np.logical_and(matrix[:, i], matrix[:, j]))
            union = np.sum(np.logical_or(matrix[:, i], matrix[:, j]))
            
            if union != 0:
                jaccard = intersection / union
                if jaccard >= s:
                    jaccard_matrix.append([i, j, jaccard])

    return jaccard_matrix

c = calculate_jaccard_difference(np.array(v), 0.3)
c


[[5, 34, 0.42424242424242425],
 [5, 55, 0.42424242424242425],
 [5, 71, 0.45161290322580644],
 [5, 91, 0.4375],
 [5, 116, 0.46875],
 [5, 130, 0.32558139534883723],
 [12, 29, 0.32558139534883723],
 [13, 75, 0.34782608695652173],
 [13, 106, 0.4],
 [32, 44, 0.3783783783783784],
 [32, 111, 0.32],
 [32, 120, 0.3783783783783784],
 [34, 55, 0.30434782608695654],
 [34, 71, 0.3181818181818182],
 [34, 91, 0.3111111111111111],
 [34, 116, 0.30434782608695654],
 [34, 130, 0.42857142857142855],
 [44, 120, 0.3888888888888889],
 [46, 56, 0.3137254901960784],
 [53, 114, 0.3333333333333333],
 [55, 71, 0.3181818181818182],
 [55, 91, 0.3111111111111111],
 [55, 116, 0.30434782608695654],
 [71, 91, 0.32558139534883723],
 [71, 116, 0.3181818181818182],
 [91, 116, 0.3111111111111111],
 [91, 130, 0.3018867924528302]]

In [125]:
par_sin_nombres[5]

'q mas claro a votar rechazo'

In [126]:
par_sin_nombres[34]

'toda la razon y para no quede mas la caga a votar rechazo'